If you're running this notebook locally on your own machine, you can ignore the following cell. If you want to [run this notebook remotely on Google's Colab platform](https://colab.research.google.com/github/tsafavi/codex/blob/master/Explore%20CoDEx.ipynb), uncomment and run the lines in the following cell:

In [4]:
!git clone https://github.com/tsafavi/codex
%cd codex
!./extract.sh en  # unzip all KG text in English
!pip install -r requirements.txt
!pip install -e .

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: data/entities/en/extracts/Q326653.txt  
  inflating: data/entities/en/extracts/Q238410.txt  
  inflating: data/entities/en/extracts/Q167543.txt  
  inflating: data/entities/en/extracts/Q453388.txt  
  inflating: data/entities/en/extracts/Q97231.txt  
  inflating: data/entities/en/extracts/Q101860.txt  
  inflating: data/entities/en/extracts/Q85399.txt  
  inflating: data/entities/en/extracts/Q670277.txt  
  inflating: data/entities/en/extracts/Q1628544.txt  
  inflating: data/entities/en/extracts/Q177883.txt  
  inflating: data/entities/en/extracts/Q65530.txt  
  inflating: data/entities/en/extracts/Q763392.txt  
  inflating: data/entities/en/extracts/Q70720.txt  
  inflating: data/entities/en/extracts/Q165592.txt  
  inflating: data/entities/en/extracts/Q966669.txt  
  inflating: data/entities/en/extracts/Q103374.txt  
  inflating: data/entities/en/extracts/Q463303.txt  
  inflating: data/entities/en/extracts/Q703778.txt  
  inflating: data/enti

If you're running on Colab, you will need to restart the runtime after running the previous cell.

In [5]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from codex.codex import Codex

In [6]:
sizes = Codex.SIZES
codes = Codex.CODES

In [7]:
sizes


['s', 'm', 'l']

# Data exploration
Load our data and do basic exploratory analysis.

In [67]:
for size in sizes:
    codex = Codex(size=size)

    train, valid, test = [
        codex.split(split) for split in ("train", "valid", "test")]
    triples = codex.triples()

    print(codex.name())

    print(
        " ", len(codex.entities()), "entities /",
        len(codex.relations()), "relations"
    )

    print(
        " ", len(train), "train /",
        len(valid), "validation /",
        len(test), "test"
    )
    print(" ", len(triples), "total triples")

CoDEx-S
  2034 entities / 42 relations
  32888 train / 1827 validation / 1828 test
  36543 total triples
CoDEx-M
  17050 entities / 51 relations
  185584 train / 10310 validation / 10311 test
  206205 total triples
CoDEx-L
  77951 entities / 69 relations
  551193 train / 30622 validation / 30622 test
  612437 total triples


In [68]:
codex = Codex(size='m')
train, valid, test = [
        codex.split(split) for split in ("train", "valid", "test")]
print(
    " ", len(codex.entities()), "entities /",
    len(codex.relations()), "relations"
)

print(
    " ", len(train), "train /",
    len(valid), "validation /",
    len(test), "test"
)

  17050 entities / 51 relations
  185584 train / 10310 validation / 10311 test


In [94]:
train_set = train.apply(lambda row: "\t".join([row['head'], row['relation'], row['tail']]), axis=1).tolist()
valid_set = valid.apply(lambda row: "\t".join([row['head'], row['relation'], row['tail']]), axis=1).tolist()
test_set = test.apply(lambda row: "\t".join([row['head'], row['relation'], row['tail']]), axis=1).tolist()

In [96]:
split_dict = {'train': train_set, 'valid': valid_set, 'test': test_set}

for name, data in split_dict.items():
  with open(f"{name}.txt", "w") as f:
    for i in range(len(data)):
      if i != len(data) - 1:
        f.write(data[i] + "\n")
      else:
        f.write(data[i])

In [114]:
import os
import json
with open("./data/entities/en/entities.json", "r") as f:
    all_entity_data = json.load(f)

In [116]:
entity_data = {}
for entity in codex.entities():
  entity_data[entity] = all_entity_data[entity]

assert len(entity_data) == len(codex.entities())

In [118]:
with open("entity2text.json", "w") as f:
    json.dump(entity_data, f, ensure_ascii=False, indent=4)

In [119]:
with open("./data/relations/en/relations.json", "r") as f:
    all_rels_data = json.load(f)

In [123]:
rel_data = {}
for rel in codex.relations():
  rel_data[rel] = all_rels_data[rel]

assert len(rel_data) == len(codex.relations())

with open("codexm.json", "w") as f:
    json.dump(rel_data, f, ensure_ascii=False, indent=4)